<a href="https://colab.research.google.com/github/lokeshvirani/Internship-Project/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1  **Import Libraries**




In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.datasets import mnist
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import euclidean_distances


In [ ]:
# Step 1 Load and preprocess MNIST dataset
def load_and_preprocess_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    # Normalize and reshape the dataset
    x_train = np.expand_dims(x_train, axis=-1) / 255.0
    x_test = np.expand_dims(x_test, axis=-1) / 255.0

    # Resize images to match ResNet50 compatability (expected shape 32, 32, 3)
    x_train = tf.image.resize(x_train, (32, 32))
    x_test = tf.image.resize(x_test, (32, 32))

    # Convert to RGB by repeating the grayscale channel three times
    x_train = np.repeat(x_train, 3, axis=-1)    #convert (32, 32 ,3)
    x_test = np.repeat(x_test, 3, axis=-1)      #convert (32, 32 ,3)

    return (x_train, y_train), (x_test, y_test)


    # Initialize ResNet50 as a feature extractor
def initialize_feature_extractor():
    resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
    resnet_model.trainable = False  # Freeze the model layers
    return resnet_model

   #Feature extraction using ResNet50
def extract_features(model, data):
    return model.predict(data, verbose=1)

# Step 2: Initialize ResNet50 as a feature extractor
def initialize_feature_extractor():
    resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
    resnet_model.trainable = False  # Freeze the model layers
    return resnet_model

# Step 3: Feature extraction using ResNet50
def extract_features(model, data):
    return model.predict(data, verbose=1)





